In [1]:
import pandas as pd
import requests
import shutil


In [2]:
# compactar dados.csv pois o arquivo era muito grande
shutil.make_archive('final', 'zip', './app', 'dados.csv')

'c:\\Users\\julia.sbardelatti\\Downloads\\rpa_project\\final.zip'

In [3]:
# descompactar arquivo dados.csv

#z = ZipFile('final.zip', 'r')
#z.extractall()
#z.close()

In [13]:
##carregando dados -> https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time_changesOnly.csv.gz

df = pd.read_csv('./app/dados.csv')
df.rename(columns={"newDeaths": "new_deaths", "newCases": "new_cases", "totalCases" : "total_cases"}, inplace=True)

print(df.shape)
df.head()

(1659310, 15)


,epi_week,date,country,state,city,ibgeID,new_deaths,deaths,new_cases,total_cases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,last_info_date
0,9,2020-02-25,Brazil,SP,São Paulo/SP,3550308,0,0,1,1,0.0,0.008,0.0,SES,2021-12-06
1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,0,0,1,1,0.0,0.000,0.0,NaN,NaN
2,9,2020-02-28,Brazil,SP,São Paulo/SP,3550308,0,0,1,2,0.0,0.016,0.0,SES,2021-12-06
3,9,2020-02-28,Brazil,TOTAL,TOTAL,0,0,0,1,2,0.0,0.001,0.0,NaN,NaN
4,10,2020-03-04,Brazil,SP,São Paulo/SP,3550308,0,0,1,3,0.0,0.024,0.0,SES,2021-12-06


### Teste 

- Teste com a data do dia 31/03/2022 para comparação com dados do Google

In [14]:
#filtrando os dados do dia 31/03/2022

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
dados_31_03_2022 = df.loc[(df['date'] == '2022-03-31')]

In [58]:
#agrupando os dados do dia 31/03/2022 por estados do Brasil, somando o total de mortes
dataframe_total_casos = dados_31_03_2022.groupby('state', as_index =False)['deaths'].sum()
dataframe_total_casos.tail()

,state,deaths
21,SC,18170
22,SE,3061
23,SP,146272
24,TO,2608
25,TOTAL,660022


## Preparando resultado final

In [5]:
df['data_referencia'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.shape

(1659310, 16)

In [6]:
df_final = df.groupby(['data_referencia','state']).sum()[["deaths", "total_cases", "new_deaths", "new_cases"]]
print(df_final.shape)

(20536, 4)


In [7]:
df_final = df_final.reset_index()
df_final = df_final.sort_values(by='data_referencia')

In [10]:
df_estados = df_final[df_final["state"].str.contains("TOTAL")==False]
df_totais = df_final[df_final["state"].str.contains("TOTAL")==True]


In [19]:
import requests

url = "https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time_changesOnly.csv.gz"
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)